# Bibliotecas e chaves

In [23]:
import os
from dotenv import load_dotenv
from llama_index.llms.groq import Groq
from llama_index.core import PromptTemplate
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings



In [3]:
load_dotenv()
GROQ_API = os.environ.get("GROQ_API")

# Inferência simples

In [5]:
#Definindo o modelo que vamos usar
llm = Groq(model='llama3-70b-8192', api_key=GROQ_API)

In [7]:
response = llm.complete('Qual é o treino usado pelo personagem Saitama de One Punch Man')

In [8]:
response

CompletionResponse(text='Uma pergunta clássica!\n\nNo anime e mangá One Punch Man, o personagem Saitama é conhecido por sua habilidade sobre-humana de derrotar qualquer oponente com apenas um golpe. Embora sua habilidade seja apresentada como uma habilidade natural, Saitama revela que alcançou esse nível de força através de um treinamento intensivo e rigoroso.\n\nO treinamento de Saitama é conhecido como "100 Push-ups, 100 Squats, 100 Sit-ups, e uma corrida de 10 km todos os dias". Sim, você leu certo! Saitama realiza esses exercícios todos os dias, sem falta, para manter e melhorar sua força e resistência.\n\nEssa rotina de treinamento é apresentada como uma piada, pois é uma combinação de exercícios comuns e uma corrida longa, mas não é nada especial ou mágico. No entanto, é a dedicação e a consistência de Saitama em realizar esses exercícios todos os dias que o tornam tão forte.\n\nAlém disso, Saitama também é apresentado como alguém que não tem interesse em melhorar suas habilidade

In [9]:
paragrafos = response.text.split("\n\n")
for paragrafo in paragrafos:
    print(paragrafo)
    print()

Uma pergunta clássica!

No anime e mangá One Punch Man, o personagem Saitama é conhecido por sua habilidade sobre-humana de derrotar qualquer oponente com apenas um golpe. Embora sua habilidade seja apresentada como uma habilidade natural, Saitama revela que alcançou esse nível de força através de um treinamento intensivo e rigoroso.

O treinamento de Saitama é conhecido como "100 Push-ups, 100 Squats, 100 Sit-ups, e uma corrida de 10 km todos os dias". Sim, você leu certo! Saitama realiza esses exercícios todos os dias, sem falta, para manter e melhorar sua força e resistência.

Essa rotina de treinamento é apresentada como uma piada, pois é uma combinação de exercícios comuns e uma corrida longa, mas não é nada especial ou mágico. No entanto, é a dedicação e a consistência de Saitama em realizar esses exercícios todos os dias que o tornam tão forte.

Além disso, Saitama também é apresentado como alguém que não tem interesse em melhorar suas habilidades ou aprender novas técnicas, poi

# Prompt Template

In [11]:
text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)

In [12]:
text_qa_template = PromptTemplate(text_qa_template_str)
text_qa_template

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Context information is below.\n---------------------\n{context_str}\n---------------------\nUsing both the context information and also using your own knowledge, answer the question: {query_str}\nIf the context isn't helpful, you can also answer the question on your own.\n")

In [13]:
refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)


In [14]:
refine_template = PromptTemplate(refine_template_str)

In [16]:
!mkdir data\paul_graham
!curl -o data\paul_graham\paul_graham_essay.txt https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 75042  100 75042    0     0   256k      0 --:--:-- --:--:-- --:--:--  260k


In [19]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [24]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="TencentBAC/Conan-embedding-v1"
)

c:\Users\Allan\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Allan\AppData\Local\llama_index\models--TencentBAC--Conan-embedding-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Allan\AppData\Local\Programs\Python\Python312\Lib\site-packages\hugg

In [25]:
index = VectorStoreIndex.from_documents(documents,llm=llm)

In [26]:
#Antes de adicionar templates
print(index.as_query_engine(llm=llm).query("Who is Joe Biden?"))

There is no mention of Joe Biden in the provided context information.


In [27]:
#Depois de adicionar templates
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("Who is Joe Biden?")
)

The context information provided does not mention Joe Biden at all. It appears to be an essay or article written by Paul Graham, a well-known entrepreneur and investor, discussing his experiences and thoughts on startups, Y Combinator, and his personal life.

Using my own knowledge, I can tell you that Joe Biden is the 46th and current President of the United States. He was inaugurated on January 20, 2021, and previously served as the 47th Vice President of the United States from 2009 to 2017.
